<a href="https://colab.research.google.com/github/Charl35X/CSC411-Mini-Project/blob/consumer-producer/Official_Producer_Consumer_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import threading
import random
import xml.etree.ElementTree as ET
# ITstudent class to store student information
class ITstudent:
    def __init__(self, name, student_id, program, courses, marks):
        self.name = name
        self.student_id = student_id
        self.program = program
        self.courses = courses
        self.marks = marks


# Buffer class to handle shared buffer
class Buffer:
    def __init__(self):
        self.buffer = []
        self.max_size = 10
        self.empty = threading.Semaphore(0)
        self.full = threading.Semaphore(self.max_size)
        self.mutex = threading.Semaphore(1)

    def insert(self, item):
        self.full.acquire()
        self.mutex.acquire()
        self.buffer.append(item)
        self.mutex.release()
        self.empty.release()

    def remove(self):
        self.empty.acquire()
        self.mutex.acquire()
        item = self.buffer.pop(0)
        self.mutex.release()
        self.full.release()
        return item

# Producer function to generate student information and XML files
def producer(buffer):
    for i in range(1, 11):
        name = "Student " + str(i)
        student_id = str(random.randint(10000000, 99999999))
        program = random.choice(["Software Engineering", "Computer Science", "Information Technology"])
        courses = ["OOP", "Data Structures", "Algorithms", "Web Development", "Databases"]

        marks = [random.randint(0, 100) for _ in range(len(courses))]
        student = ITstudent(name, student_id, program, courses, marks)

        # Generate XML file
        root = ET.Element("Student")
        ET.SubElement(root, "Name").text = student.name
        ET.SubElement(root, "StudentID").text = student.student_id
        ET.SubElement(root, "Program").text = student.program
        courses_elem = ET.SubElement(root, "Courses")
        for course, mark in zip(student.courses, student.marks):
            course_elem = ET.SubElement(courses_elem, "Course")
            ET.SubElement(course_elem, "Name").text = course
            ET.SubElement(course_elem, "Mark").text = str(mark)

        tree = ET.ElementTree(root)
        tree.write("student{}.xml".format(i))

        # Insert file number into buffer
        buffer.insert(i)

# Consumer function to read XML files, calculate average marks, and print student information
def consumer(buffer):
    while True:
        file_number = buffer.remove()
        tree = ET.parse("student{}.xml".format(file_number))
        root = tree.getroot()

        name = root.find("Name").text
        student_id = root.find("StudentID").text
        program = root.find("Program").text
        courses = []
        marks = []
        for course_elem in root.find("Courses"):
            course = course_elem.find("Name").text
            mark = int(course_elem.find("Mark").text)
            courses.append(course)
            marks.append(mark)

        student = ITstudent(name, student_id, program, courses, marks)

        # Calculate average mark
        average_mark = sum(student.marks) / len(student.marks)

        # Determine pass/fail
        pass_fail = "Pass" if average_mark >= 50 else "Fail"

        # Print student information
        print("Student Name:", student.name)
        print("Student ID:", student.student_id)
        print("Program:", student.program)
        print("Courses and Marks:")
        for course, mark in zip(student.courses, student.marks):
            print(course, "-", mark)
        print("Average Mark:", average_mark)
        print("Pass/Fail:", pass_fail)
        print()

# Create buffer and start producer and consumer threads
buffer = Buffer()
producer_thread = threading.Thread(target=producer, args=(buffer,))
consumer_thread = threading.Thread(target=consumer, args=(buffer,))
producer_thread.start()
consumer_thread.start()

Student Name: Student 1
Student ID: 25282728
Program: Software Engineering
Courses and Marks:
OOP - 98
Data Structures - 12
Algorithms - 77
Web Development - 96
Databases - 95
Average Mark: 75.6
Pass/Fail: Pass

Student Name: Student 2
Student ID: 82756727
Program: Information Technology
Courses and Marks:
OOP - 97
Data Structures - 26
Algorithms - 95
Web Development - 83
Databases - 86
Average Mark: 77.4
Pass/Fail: Pass

Student Name: Student 3
Student ID: 94235285
Program: Information Technology
Courses and Marks:
OOP - 15
Data Structures - 35
Algorithms - 34
Web Development - 75
Databases - 64
Average Mark: 44.6
Pass/Fail: Fail

Student Name: Student 4
Student ID: 37429496
Program: Computer Science
Courses and Marks:
OOP - 70
Data Structures - 51
Algorithms - 30
Web Development - 23
Databases - 28
Average Mark: 40.4
Pass/Fail: Fail

Student Name: Student 5
Student ID: 28428173
Program: Computer Science
Courses and Marks:
OOP - 30
Data Structures - 95
Algorithms - 0
Web Development -